In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
from pathlib import Path
from collections import Counter
import json
import nltk

nltk.download("punkt")
nltk.download("stopwords")

tf_idf = TfidfVectorizer(stop_words="english")


def encode_topics(df):
    # topics = df["topics"].str.get_dummies(sep=",")
    # topics = df["topics"].apply( topicfor topic in topics  )
    one_hot_encoded = (
        pd.get_dummies(df["topics"].apply(pd.Series).stack()).groupby(level=0).sum()
    )
    df = pd.concat([df, one_hot_encoded], axis=1)
    # print(df)
    return df


def set_index(df, index_column="poll_ID"):
    df.set_index(index_column, inplace=True)
    return df


def reset_index(df):
    df.reset_index()
    return df


def check_column_type(df, column_name, check_type):
    column_index = df.columns.get_loc(column_name)
    for i in range(len(df)):
        if not isinstance(df.iloc[i, column_index], check_type):
            print(
                f"error: {df.iloc[i, 0], df.iloc[i, 1],df.iloc[i, 2], df.iloc[i, 3], df.iloc[i, 4]}"
            )


def preprocess_text(text):
    tokens = nltk.tokenize.word_tokenize(text)
    # tokens = [word.lower() for word in tokens if type(word) is str]
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word not in string.punctuation]
    stop_words = set(nltk.corpus.stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]
    processed_text = " ".join(tokens)

    return processed_text


def preprocess_list(field_list):
    ret_list = []
    stop_words = set(nltk.corpus.stopwords.words("english"))
    for item in field_list:
        tokens = nltk.tokenize.word_tokenize(item)
        # tokens = [word.lower() for word in tokens if type(word) is str]
        tokens = [word.lower() for word in tokens]
        tokens = [word for word in tokens if word not in string.punctuation]
        tokens = [word for word in tokens if word not in stop_words]
        processed_text = " ".join(tokens)
        ret_list.append(processed_text)

    return ret_list


def create_tf_idf_matrix(df, column):
    # print(f"{df[column]} is {df[column].dtype} and {df[column].dtype is list} {list}: ")
    df[column] = df[column].apply(lambda x: " ".join(x))
    df[column] = df[column].apply(preprocess_text)

    return tf_idf.fit_transform(df[column])


def create_souped_tf_idf_matrix(df):
    df["topics"] = df["topics"].apply(preprocess_list)
    df["question"] = df["question"].apply(preprocess_text)

    # Create a new soup feature
    df["soup"] = df.apply(create_soup, axis=1)

    return tf_idf.fit_transform(df["soup"])


def create_soup(df):
    res = (
        df["question"]
        + " "
        + " ".join(df["options"])
        + " "
        + (4 * (" " + " ".join(df["topics"])))
    )
    # print(f"-----------------------------------\n* Processing: [{ }]")
    return res


def calc_cosine_similarity_matrix(tf_idf_matrix_1, tf_idf_matrix_2):
    return cosine_similarity(tf_idf_matrix_1, tf_idf_matrix_2)


def id_to_index(df, id):
    return df[df["id"] == id].index.values[0]


def title_from_idx(df, idx):
    return df[df.index == idx]


def gen_recommendations(
    index,
    df,
    cosine_similarity_matrix,
    number_of_recommendations,
):
    # index = idx_from_title(df, original_title)
    similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
    similarity_scores_sorted = sorted(
        similarity_scores, key=lambda x: x[1], reverse=True
    )

    recommendations_indices = [
        t[0] for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
    ]
    recommendations = list(df["title"].iloc[recommendations_indices])
    # print(recommendations)
    # print(similarity_scores_sorted, type(similarity_scores_sorted))
    # recommendations_indices = [
    #    t[0] for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
    # ]
    # recommendations_scores = [
    #    t[1] for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
    # ]
    # return (df["title"].iloc[recommendations_indices], recommendations_scores)

    return recommendations


def gen_rec_from_list_of_polls(
    interacted_polls, polls, cosine_similarity_matrix, number_of_recommendations
):
    recommendations = []
    for poll_id in interacted_polls:
        index = id_to_index(polls, poll_id)
        similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
        similarity_scores_sorted = sorted(
            similarity_scores, key=lambda x: x[1], reverse=True
        )

        recommendations_indices = [
            t[0] for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
        ]
        recs = list(polls["id"].iloc[recommendations_indices])

        # Filter out polls that have already been interacted with
        filtered_recs = [poll for poll in recs if poll not in interacted_polls]

        recommendations.append(filtered_recs)

    flattened_recommendations = [
        item for sublist in recommendations for item in sublist
    ]
    flattened_recommendations = Counter(flattened_recommendations)
    n_most_recommended = flattened_recommendations.most_common(
        number_of_recommendations
    )
    n_most_recommended = [t[0] for t in n_most_recommended]
    # print(n_most_recommended)

    return n_most_recommended


[nltk_data] Downloading package punkt to /home/erfan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/erfan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from elasticsearch import Elasticsearch
import json
class ElasticsearchHandel:
    def __init__(self, elasticsearch_url, username, password, fingerprint):
        self.elasticsearch_url = elasticsearch_url
        self.username = username
        self.password = password
        self.fingerprint = fingerprint
        self.client = Elasticsearch(
            hosts=self.elasticsearch_url,
            basic_auth=(self.username, self.password),
            ssl_assert_fingerprint=self.fingerprint,
        )

    def get_index(self, index_name, batch_size=100):
        setattr(self, index_name, [])
        index_list = getattr(self, index_name)
        from_index = 0
        all_instances = []

        while True:
            # query = {"query": {"match_all": {}}, "size": batch_size, "from": from_index}
            results = self.client.search(
                index=index_name,
                query={"match_all": {}},
                size=batch_size,
                from_=from_index,
            )
            instances = results["hits"]["hits"]

            all_instances.extend(instances)
            from_index += batch_size
            if len(instances) < 100:
                break

        setattr(self, index_name, [instance["_source"] for instance in all_instances])
        return getattr(self, index_name)

    def get_interactions(self, index_name, user_id, batch_size=100):
        # setattr(self, index_name, [])
        # index_list = getattr(self, index_name)
        from_index = 0
        all_instances = []

        query = {
            "match_phrase": {"userId": user_id},
        }

        results = self.client.search(
            index=index_name,
            query=query,
            size=batch_size,
            from_=from_index,
            timeout="1s",
        )
        # instances = results["hits"]["hits"][0]
        hits = results["hits"].get("hits")

        if not hits:
            # raise ValueError("User doesn't have any interactions.")
            raise InteractionNotFound()

        return hits[0].get("_source")

    def get_trend_polls(self, polls):
        # polls = getattr(self, "polls")
        # trend_polls = sorted(polls, key=lambda x: (-x["numberOfPollups"], -x["numberOfVotes"], -x["numberOfLike"]))
        trend_polls = sorted(
            polls,
            key=lambda x: (
                -x["numberOfVotes"],
                -x["numberOfLike"],
                # -x["numberOfPollUp"],
            ),
        )

        # recs = trend_polls["id"]

        # print("\n", filtered_trend_polls, "\n")
        # setattr(self, "trend_polls", trend_polls)
        return trend_polls

    def export_index_to_file(self, index, index_file_path):
        try:
            with open(index_file_path, "w") as output:
                # for instance in self.instances:
                #        json.dump(instance["_source"], output, indent=4)
                json.dump(index, output, indent=4)
        except Exception as exp:
            print("Export Error", exp)


In [6]:
import pandas as pd

elasticsearch_url = "https://159.203.183.251:9200"
username = "pollett"
password = "9r0&rJP@19GY"
fingerprint = "CE:AA:F7:FF:04:C7:31:14:78:9C:62:D4:CE:98:F9:EF:56:DA:70:45:37:14:E3:F8:66:0A:25:ED:05:04:83:ec"


elastic_handle = ElasticsearchHandel(
    elasticsearch_url, username, password, fingerprint
)

polls = elastic_handle.get_index("polls")
trend_polls = elastic_handle.get_trend_polls(polls)

polls_df = pd.DataFrame.from_records(polls)

polls_df

,id,question,options,topics,pollType,ownerId,createdAt,numberOfLike,numberOfViews,numberOfVotes,numberOfComments,numberOfPollUp,pollLimitations,isDeleted
0,c076c4b5-e3ae-4ca5-8191-00de0628a326,Do you research pet-friendly attractions and a...,"[Always, Rarely, Sometimes]","[Pets, Travel]",Public,a55320db-8ada-4e1f-8cc1-f8d5b52bbc0c,2023-09-28T09:52:15.043355Z,0,0,0,0,0,NaN,NaN
1,c1a67395-ce58-4df5-993a-cf48293f4513,Should athletes be required to align with cert...,"[No, Yes]","[Politics, Sport]",Public,0c6ef01a-de23-4c22-b72e-9a3f15b5fde2,2023-09-28T09:52:15.051446Z,0,0,0,0,0,NaN,NaN
2,c220777e-4a39-4401-a8b9-fe648fa8a6fc,Do you believe politicians try to exert contro...,"[No, Yes]","[Politics, Sport]",Public,0c6ef01a-de23-4c22-b72e-9a3f15b5fde2,2023-09-28T09:52:15.051441Z,0,0,0,0,0,NaN,NaN
3,c220fe81-7b78-4c89-b87e-d8c80cb80c67,Which paint finish provides the best resistanc...,"[Eggshell, Matte, Semi-gloss]","[Home Décor, Science]",Public,72edb639-4231-44d4-92a2-ac7310bd6745,2023-09-28T09:52:15.043387Z,0,0,0,0,0,NaN,NaN
4,c391295a-2629-43a3-b554-8f8140ba44a4,Have you ever taken your pet on a road trip?,"[No, Yes, Planning to in the future]","[Pets, Travel]",Public,a55320db-8ada-4e1f-8cc1-f8d5b52bbc0c,2023-09-28T09:52:15.04336Z,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2113,ffc775ab-6034-4454-9ddc-ca3b032a64a8,Which drink would you order at a fashion event...,"[Signature cocktail, Champagne]","[Food&Drinks, Fashion]",Public,99a9c3cc-27cf-4fc2-bce8-899c61c1a152,2023-09-28T09:52:15.045845Z,0,0,0,0,0,"{'allowedLocations': [], 'allowedGender': 'All...",False
2114,fff702c2-fe64-46ff-b62d-757a8ca87e32,Which type of faucet material is most resistan...,"[Brass, Nickel, Stainless steel]","[Home Décor, Science]",Public,3ae14a1d-d340-45d1-a796-f66d901c35aa,2023-09-28T09:52:15.043532Z,0,0,0,0,0,"{'allowedLocations': [], 'allowedGender': 'All...",False
2115,c6c7e104-de7c-4889-a5ad-e0afda2ac830,Which state you like?,"[NY, GA, CA, GA, LA, VA, FL]",[General],Public,2e4903c8-99b9-4606-8534-a9996e37ed22,0001-01-01T00:00:00,0,0,0,0,0,NaN,NaN
2116,50fbde91-0af5-49ea-bf21-db7643699ad2,gdft,"[g, g]",[General],Public,e00b366a-37a8-407d-9a15-e585d1ad539a,0001-01-01T00:00:00,0,0,0,0,0,NaN,NaN


In [7]:
test_limitations = {
  "allowedLocations": [ {"country": "United States", "city": "United States", "state": "NJ"},
                        {"country": "United States", "city": "United States", "state": "NJ"},
                        {"country": "United States", "city": "United States", "state": "NJ"}],
  "allowedGender": "All",
  "allowedAgeRange": {"minimumAge": 0, "maximumAge": 120},
  "isDeleted": False
  }

user_limitations = {
    "Location":{"country": "United States", "city": "United States", "state": "NJ"},
    "Gender": "Men",
    "Age": 30
}


In [58]:

#def meets_limitations(poll, user):
#    poll_limitations = poll['poll_limitations']
#    is_deleted = poll['isDeleted']
#    if is_deleted:
#        return False 
#    print(f"Poll_limitations: {poll_limitations}")
#    print(f"User: {user}")
#    
#    # Your conditions here
#    
#    if pd.notna(poll_limitations):
#        # Condition 1
#        print(f"Condition 1: {user.get('Location')} not in {poll_limitations.get('allowedLocations', [])}")
#        if 'Location' in user and 'allowedLocations' in poll_limitations:
#            if all(location != user.get('Location') for location in poll_limitations.get("poll_limitations")):
#                return False
#        
#        # Condition 2
#        print(f"Condition 2: {poll_limitations['allowedGender']} != 'All' and {user['Gender']} != {poll_limitations['allowedGender']}")
#        if 'Gender' in user and 'allowedGender' in poll_limitations:
#            if poll_limitations.get('allowedGender') != 'All' and user.get('Gender') != poll_limitations.get('allowedGender'):
#                return False
#        
#        # Condition 3
#        if 'Age' in user and 'allowedAgeRange' in poll_limitations:
#            age = user['Age']
#            min_age = poll_limitations['allowedAgeRange'].get('minimumAge', 0)
#            max_age = poll_limitations['allowedAgeRange'].get('maximumAge', 120)
#            print(f"Condition 3: Age: {user['Age']}, Min: {min_age}, Max: {max_age}")
#            if age < min_age or age > max_age:
#                return False
#        
#    return False

# Function to check if the values are in the expected format
def is_valid_limitations(limitations):
        return (
            'allowedLocations' in limitations and
            'allowedGender' in limitations and
            'allowedAgeRange' in limitations
        )
# Function to filter polls with user-defined limitations
#def filter_polls(row, user_limitations):
#    if is_valid_limitations(row.get('pollLimitations')) and all(k in user_limitations for k in ['Location', 'Gender', 'Age']):
#        user_location = user_limitations.get('Location')
#        
#        allowed_locations = row.get('pollLimitations').get('allowedLocations')
#        if len(allowed_locations) != 0:
#                
#            if any(user_location == loc for loc in allowed_locations):
#                allowed_gender = row['pollLimitations']['allowedGender']
#                user_gender = user_limitations['Gender']
#                if allowed_gender == 'All' or allowed_gender == user_gender:
#                    allowed_age_range = row['pollLimitations']['allowedAgeRange']
#                    user_age = user_limitations['Age']
#                    if allowed_age_range['minimumAge'] <= user_age <= allowed_age_range['maximumAge']:
#                        return True
#    return False
def filter_polls(row, user_limitations):
    print(isinstance(row.get('pollLimitations'), dict))
    if isinstance(row.get('pollLimitations'), dict) and all(k in user_limitations for k in ['Location', 'Gender', 'Age']):
        user_location = user_limitations.get('Location')
        
        allowed_locations = row.get('pollLimitations').get('allowedLocations')
        if len(allowed_locations) == 0 or any(user_location == loc for loc in allowed_locations):
            allowed_gender = row['pollLimitations']['allowedGender']
            user_gender = user_limitations['Gender']
            if allowed_gender == 'All' or allowed_gender == user_gender:
                allowed_age_range = row['pollLimitations']['allowedAgeRange']
                user_age = user_limitations['Age']
                if allowed_age_range['minimumAge'] <= user_age <= allowed_age_range['maximumAge']:
                    print("All conditions met. Returning True")
                    return True
                else:
                    print("Age condition not met.")
            else:
                print("Gender condition not met.")
            
        else:
            print("No allowedLocations found.")
    else:
        print("Invalid limitations or missing keys in user_limitations.")
    return False

In [60]:
# Filter the DataFrame based on the user's limitations
#filtered_df = polls_df[polls_df['pollLimitations'].apply(lambda x: meets_limitations(x, test_user))]
filtered_df = polls_df[polls_df.apply(filter_polls, args=(user_limitations,), axis=1)]


False
Invalid limitations or missing keys in user_limitations.
False
Invalid limitations or missing keys in user_limitations.
False
Invalid limitations or missing keys in user_limitations.
False
Invalid limitations or missing keys in user_limitations.
False
Invalid limitations or missing keys in user_limitations.
False
Invalid limitations or missing keys in user_limitations.
False
Invalid limitations or missing keys in user_limitations.
False
Invalid limitations or missing keys in user_limitations.
False
Invalid limitations or missing keys in user_limitations.
False
Invalid limitations or missing keys in user_limitations.
False
Invalid limitations or missing keys in user_limitations.
False
Invalid limitations or missing keys in user_limitations.
False
Invalid limitations or missing keys in user_limitations.
False
Invalid limitations or missing keys in user_limitations.
False
Invalid limitations or missing keys in user_limitations.
False
Invalid limitations or missing keys in user_limit

In [61]:
filtered_df

,id,question,options,topics,pollType,ownerId,createdAt,numberOfLike,numberOfViews,numberOfVotes,numberOfComments,numberOfPollUp,pollLimitations,isDeleted
158,1edc7b0b-df6b-439f-81ee-fc654f3ce4f6,Wich one?,"[Sketch, XD, Figma]","[Tech, Sport]",Public,4c208829-0b25-47a6-8574-b10fc3db7ab2,2023-05-07T06:24:48.218491Z,2,7,4,0,1,"{'allowedGender': 'All', 'allowedAgeRange': {'...",False
159,1ec1f108-2c7b-45d5-819c-9db42ff7a9ab,Which one ?,"[one, two]","[Activity, Tech]",Public,66271a97-73ba-41c8-b460-23d166e4c020,2023-04-27T09:06:50.953044Z,2,3,0,0,0,"{'allowedGender': 'All', 'allowedAgeRange': {'...",False
160,384378fe-01e5-40d9-90cb-17c78844250f,What is your favorite musical genre?,"[Rock, Jazz, Pop, Classic]","[Music, Art]",Private,4a5f6a4d-f482-406b-9d81-fb9586b580c0,2023-05-08T06:40:05.075076Z,1,3,1,0,0,"{'allowedGender': 'All', 'allowedAgeRange': {'...",False
161,3c269d38-d5c0-4e94-b101-a6d745a6bc66,Which on?,"[Taskoloo, Trello, Click up]","[Sport, Activity, Sport, Activity, Activity]",Public,4c208829-0b25-47a6-8574-b10fc3db7ab2,2023-05-04T09:56:55.615412Z,2,4,4,0,0,"{'allowedGender': 'All', 'allowedAgeRange': {'...",False
162,4230b1ae-a544-4bea-9603-e1cb08954f5c,which animal do you have?,"[Cat , Dog]",[],Public,4c208829-0b25-47a6-8574-b10fc3db7ab2,2023-04-30T08:05:21.159336Z,1,3,3,0,0,"{'allowedGender': 'All', 'allowedAgeRange': {'...",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2110,ff1bf610-2a2c-4702-9d79-ed8e045dfc0e,Is your fashion style more influenced by Frenc...,"[Bohemian vibes, French chic]",[Fashion],Public,99a9c3cc-27cf-4fc2-bce8-899c61c1a152,2023-09-28T09:52:15.045843Z,0,0,0,0,0,"{'allowedLocations': [], 'allowedGender': 'All...",False
2111,ffa5480b-2af7-4f2f-acc6-72fbd9605c6e,Does playing a musical instrument have a posit...,"[Strongly Agree, Disagree, Neutral, Agree, Str...","[Music, Health]",Public,ec50681b-7f44-4a87-9070-3730171bd2c1,2023-09-28T09:52:15.044524Z,0,0,0,0,0,"{'allowedLocations': [], 'allowedGender': 'All...",False
2112,ffc2df1a-e0d4-46e8-a886-c1bb90b175c2,What do you #prefer ? #one or #two ?,"[2, 1]","[Tech, Sport, Politics]",Public,66271a97-73ba-41c8-b460-23d166e4c020,2023-05-24T12:53:02.575363Z,2,6,1,0,0,"{'allowedLocations': [], 'allowedGender': 'All...",False
2113,ffc775ab-6034-4454-9ddc-ca3b032a64a8,Which drink would you order at a fashion event...,"[Signature cocktail, Champagne]","[Food&Drinks, Fashion]",Public,99a9c3cc-27cf-4fc2-bce8-899c61c1a152,2023-09-28T09:52:15.045845Z,0,0,0,0,0,"{'allowedLocations': [], 'allowedGender': 'All...",False


In [8]:
nan_limitation_count = polls_df["pollLimitations"].isna().sum()
len(polls_df)
nan_limitation_count

157

In [9]:
polls_tf_idf_matrix = create_souped_tf_idf_matrix(polls_df)
#print(polls_tf_idf_matrix[0,1])
#print(polls_tf_idf_matrix[0].data)
#print(polls_tf_idf_matrix[0])

polls_tf_idf_matrix

<2114x5592 sparse matrix of type '<class 'numpy.float64'>'
	with 28245 stored elements in Compressed Sparse Row format>

The `polls_tf_idf_matrix` is a sparse matrix used to represent textual data in a numerical format. Let's break down its characteristics:

- **Dimensions**: The matrix has dimensions of 142 rows and 274 columns.

- **Sparse Matrix**: It's classified as a sparse matrix, meaning that the majority of its elements are zero. This is common in text data like TF-IDF matrices, where most terms do not appear in every document.

- **Data Type**: The elements of the matrix are of type `numpy.float64`, representing 64-bit floating-point numbers. This is the standard data type for TF-IDF values.

- **Stored Elements**: There are 586 non-zero elements (entries) in the matrix. Sparse matrices are memory-efficient because they only store these non-zero values.

- **Compressed Sparse Row Format (CSR)**: The matrix is stored in the Compressed Sparse Row (CSR) format, a widely used format for sparse matrices. It allows for efficient row-wise access and arithmetic operations.

In summary, the `polls_tf_idf_matrix` efficiently represents TF-IDF values of text data with 142 rows and 274 columns. Its sparse nature optimizes memory usage by storing only non-zero values, making it suitable for text analysis tasks.


In [11]:
cosine_similarity_matrix = calc_cosine_similarity_matrix(
                polls_tf_idf_matrix, polls_tf_idf_matrix
            )
cosine_similarity_matrix.shape


(2114, 2114)

**Cosine Similarity Matrix Explanation:**

- **Definition:** The `cosine_similarity_matrix` is a matrix designed to represent the similarity between pairs of polls within a dataset.

- **Calculation:** It is calculated using the `calc_cosine_similarity_matrix` function, which commonly utilizes the cosine similarity metric. Cosine similarity is a frequently used measure in natural language processing and information retrieval. It assesses the similarity between two vectors, in this context, the TF-IDF vectors representing the polls.

- **Interpretation:** In the `cosine_similarity_matrix`, each element `(i, j)` denotes the cosine similarity between two polls: poll `i` and poll `j`. The values within this matrix have a range from -1 to 1, and their meanings are as follows:

    - `1`: Indicates that the polls are identical or have the highest possible similarity.
    - `0`: Denotes that the polls are orthogonal, implying no similarity between them.
    - `-1`: Suggests that the polls are diametrically opposite or possess the highest possible dissimilarity.

This matrix is critical for generating recommendations as it quantifies the textual content's similarity or dissimilarity between different polls. By leveraging this similarity matrix, the recommendation system can identify polls with content similar to those the user has interacted with, resulting in more personalized and relevant recommendations.


In [12]:
#user_id = request.args.get("userId")
user_id = "67eb27ca-ba0b-4d29-8627-9ec78327b512"



userInteractions = elastic_handle.get_interactions(
                "userpollinteractions", user_id
            )

userInteractions = [
                interaction["pollId"]
                for interaction in userInteractions["userPollActions"][:20]
            ]
recommended_list = gen_rec_from_list_of_polls(
                userInteractions,
                polls_df,
                cosine_similarity_matrix,
                100,
            )

recommended_polls = polls_df[polls_df["id"].isin(recommended_list)]
recommended_polls

,id,question,options,topics,pollType,ownerId,createdAt,numberOfLike,numberOfViews,numberOfVotes,numberOfComments,numberOfPollUp,soup
14,c1bf5f12-8a6c-4873-997f-f3ab7e57fb79,type insulation material provides best resista...,"[Mineral wool, Cellulose, Spray foam]","[home décor, science]",Public,3ae14a1d-d340-45d1-a796-f66d901c35aa,2023-09-28T09:52:15.043557Z,0,0,0,0,0,type insulation material provides best resista...
16,c220fe81-7b78-4c89-b87e-d8c80cb80c67,paint finish provides best resistance moisture...,"[Eggshell, Matte, Semi-gloss]","[home décor, science]",Public,72edb639-4231-44d4-92a2-ac7310bd6745,2023-09-28T09:52:15.043387Z,0,0,0,0,0,paint finish provides best resistance moisture...
38,c53adab3-373a-415d-b150-c56a4ceb8633,color scheme prefer home office,"[Cool blues and grays, Warm earth tones, Vibra...",[home décor],Public,8d2908fd-e0cf-4380-a881-1917c7022ed9,2023-09-28T09:52:15.046347Z,0,0,0,0,0,color scheme prefer home office Cool blues and...
52,c6fc5bee-1c6e-4ea9-8493-ae5b88c12523,art movement find intriguing,"[Cubism, Pop Art, Surrealism, Minimalism, Roma...","[art, activity]",Public,26e4e08f-c554-4e66-a0b1-4f67e60a04dc,2023-09-28T09:52:15.050744Z,0,0,0,0,0,art movement find intriguing Cubism Pop Art Su...
63,c7b3c378-04b8-4db2-b579-b9625fd0cf4f,type insulation material environmentally frien...,"[Hemp, Recycled denim, Sheep's wool]","[home décor, science]",Public,72edb639-4231-44d4-92a2-ac7310bd6745,2023-09-28T09:52:15.043417Z,0,0,0,0,0,type insulation material environmentally frien...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2028,19475b13-504e-4650-bf53-487d7b718ca7,incorporate technology living room design,"[Motorized window shades, Smart TV, Wireless s...","[home décor, tech]",Public,8d2908fd-e0cf-4380-a881-1917c7022ed9,2023-09-28T09:52:15.04641Z,0,0,0,0,0,incorporate technology living room design Moto...
2039,349c0fd7-d326-419d-8e8d-721cdd141ff3,art style movement best represents concept min...,"[Minimalism, Abstract expressionism, Land art,...","[art, activity]",Public,67eb27ca-ba0b-4d29-8627-9ec78327b512,2023-09-28T09:52:15.050966Z,0,0,0,0,0,art style movement best represents concept min...
2074,6c17d6e8-534b-4432-8ae3-3583f5baa632,type tv stand prefer entertainment system,"[Floating wall mount, Media cabinet, TV console]","[tech, home décor]",Public,a2a14c2b-eacf-405b-8f79-ace81728ba3b,2023-09-28T09:52:15.046306Z,0,0,0,0,0,type tv stand prefer entertainment system Floa...
2080,6d269392-f854-49e7-b781-e933b6e6e59a,opinion public art contribute local economic g...,"[No, Yes]","[art, economy]",Public,ee4528c1-a1cd-4d34-946a-88094df8021c,2023-09-28T09:52:15.051082Z,0,0,0,0,0,opinion public art contribute local economic g...
